# Extração e Classificação de Dados Imunohistoquímicos de Câncer de Mama

## Introdução Técnica

Este notebook implementa um sistema de extração e classificação de dados a partir de laudos médicos de exames imunohistoquímicos de câncer de mama, utilizando modelos de linguagem de grande escala (LLMs) integrados com técnicas de processamento de linguagem natural.

### Objetivo Principal

O objetivo principal deste notebook é extrair informações estruturadas de laudos médicos não estruturados de exames imunohistoquímicos, para permitir a classificação molecular dos tumores de mama em subtipos clinicamente relevantes. Essa classificação é essencial para decisões terapêuticas e prognóstico das pacientes.

### Tecnologias Utilizadas

- **Processamento de Dados**: PySpark, pandas
- **Aprendizado de Máquina**: Modelos de linguagem natural (LLMs) via API Databricks
- **Processamento de Texto**: regex (expressões regulares)
- **Monitoramento e Rastreamento**: MLflow
- **Armazenamento de Dados**: Delta Lake
- **Notificações e Alertas**: Octoops (Sentinel)
- **Outras Bibliotecas**: numpy, json, tqdm, ast

### Fluxo de Trabalho/Etapas Principais

1. **Configuração do Ambiente**: Instalação de dependências e inicialização da sessão Spark
2. **Carregamento de Dados**: Consulta SQL para extrair laudos específicos de imunohistoquímica
3. **Preparação de Prompts**: Criação de templates para interação com o LLM
4. **Processamento de Laudos**: Envio dos laudos ao LLM para extração de informações estruturadas
5. **Extração de Biomarcadores**: Identificação de receptores hormonais (ER, PR), HER-2, Ki-67 e CK5/6
6. **Classificação Molecular**: Determinação do subtipo molecular do tumor (Luminal A, Luminal B, HER-2, Triplo Negativo)
7. **Avaliação da Qualidade**: Comparação das extrações do LLM com extrações por regex
8. **Persistência de Dados**: Salvamento dos resultados em tabela Delta para uso posterior

### Dados Envolvidos

- **Fonte de Dados**: Tabela `refined.saude_preventiva.fleury_laudos`
- **Tabela de Destino**: `refined.saude_preventiva.fleury_laudos_mama_imunohistoquimico`
- **Tipos de Exame**: Exames com siglas "IH-NEO" e "IHMAMA" (imunohistoquímica)
- **Filtros Aplicados**: 
  - `linha_cuidado = 'mama'`
  - `sexo_cliente = 'F'`
  - Presença dos termos "mama" e "carcinoma" no laudo
- **Campos Extraídos**:
  - `receptor_estrogeno`: Status do receptor de estrogênio (POSITIVO/NEGATIVO)
  - `receptor_progesterona`: Status do receptor de progesterona (POSITIVO/NEGATIVO)
  - `status_her2`: Status do receptor HER-2 (POSITIVO/INCONCLUSIVO/NEGATIVO)
  - `ki67_percentual`: Valor percentual do marcador de proliferação Ki-67
  - `status_ck5_6`: Status do marcador citoqueratina 5/6 (POSITIVO/NEGATIVO)

### Resultados/Saídas Esperadas

1. **DataFrame Enriquecido**: Laudos médicos com campos estruturados extraídos
2. **Classificação Molecular**: Cada caso classificado em um subtipo molecular:
   - Luminal A
   - Luminal B
   - Luminal com HER2 Positivo
   - HER-2 Superexpresso
   - Triplo Negativo
3. **Métricas de Qualidade**: Avaliação da precisão das extrações automáticas
4. **Persistência em Delta Lake**: Dados salvos para uso em pipelines downstream

### Pré-requisitos

- **Ambiente Databricks**: Com acesso ao endpoint do modelo de linguagem
- **Dependências Python**: openai, pandas, pyspark, mlflow, octoops
- **Permissões de Acesso**: Acesso à tabela de laudos médicos e permissão para escrita na tabela de destino
- **Token de Autenticação**: Para acesso à API do modelo de linguagem

### Considerações Importantes

- **Privacidade de Dados**: Os dados são tratados conforme políticas de privacidade médica
- **Validação Clínica**: As classificações moleculares automáticas devem ser validadas por especialistas
- **Processamento Seletivo**: Apenas laudos contendo termos de carcinoma são processados pelo LLM
- **Limitações**: A precisão da extração depende da qualidade e padronização dos laudos originais
- **Avaliação de Qualidade**: Utiliza-se regex como pseudo-gold para avaliar a qualidade da extração do LLM

# Extração de dado - Imunohistoquimico
**Extrair os seguintes labels estruturados:**
- Receptor de Estrógeno (categórica): POSITIVO ou NEGATIVO 
  - "RECEPTOR DE ESTRÓGENO - POSITIVO " 
  - "RECEPTOR DE ESTRÓGENO – NEGATIVO" 

- Receptor de Progesterona (categórica): POSITIVO ou NEGATIVO 
  - "RECEPTOR DE PROGESTERONA - POSITIVO" 
  - "RECEPTOR DE PROGESTERONA - NEGATIVO" 

- Status do HER-2 (categórica): Negativo, inconclusivo ou positivo 
  - "HER-2 - ESCORE 0" = Negativo 
  - "HER-2 - ESCORE 1+" = Negativo 
  - "HER-2  -  ESCORE  2+" = Inconclusivo 
  - "HER-2  -  ESCORE  3+" = Positivo 

- Porcentagem do Ki-67 (numérica):  "KI-67 - POSITIVO EM 20% DAS CÉLULAS NEOPLÁSICAS"
  - Deve ser extraído esse número da porcentagem nessa frase 

- Status CK5/6 (categórica): POSITIVO ou NEGATIVO 
  - "CK5/6 - POSITIVO "ou "CK5/6 - NEGATIVO" 
  
+ Para laudos que não possuem carcinoma, ou seja, casos negativos para câncer não devem se processados no LLM.

## Instalação de Dependências

Esta célula instala o pacote `octoops`, que é utilizado para monitoramento e alertas no ambiente Databricks. As outras dependências necessárias (como openai, tqdm, pandas e databricks-feature-store) estão comentadas pois provavelmente já estão instaladas no ambiente ou serão instaladas por outro processo.

O pacote Octoops permite configurar alertas e notificações em caso de falhas no pipeline, sendo uma peça importante na infraestrutura de monitoramento do processo de extração de dados.

In [ ]:
# %pip install openai -q
# %pip install tqdm -q
# %pip install pandarallel -q
# %pip install databricks-feature-store -q
# %pip install ace_tools -q
%pip install octoops

## Reinicialização do Python

Após a instalação de novas dependências, é necessário reiniciar o interpretador Python para garantir que as bibliotecas recém-instaladas estejam disponíveis no ambiente de execução. O comando `dbutils.library.restartPython()` realiza essa reinicialização, garantindo que todas as dependências estejam carregadas corretamente antes de prosseguir com a execução do notebook.

In [ ]:
dbutils.library.restartPython()

## Importação de Bibliotecas e Configuração Inicial

Esta célula realiza a importação de todas as bibliotecas necessárias para o processamento dos laudos e inicializa o ambiente Spark.

**Objetivos principais:**
1. Importar bibliotecas para manipulação de dados (PySpark, pandas, numpy)
2. Importar bibliotecas para processamento de texto e regex
3. Importar ferramentas para logging e monitoramento (mlflow)
4. Importar bibliotecas para integração com APIs externas (openai)
5. Inicializar a sessão Spark
6. Obter o token de autenticação Databricks

**Bibliotecas principais:**
- **PySpark**: Framework para processamento distribuído de dados
- **pandas/numpy**: Ferramentas para manipulação e análise de dados
- **openai**: Cliente para comunicação com APIs de modelos de linguagem
- **mlflow**: Plataforma para gerenciamento do ciclo de vida de modelos de ML
- **tqdm**: Biblioteca para barras de progresso
- **re**: Biblioteca para processamento de expressões regulares

A sessão Spark é configurada com o nome "LLM_Extractor", e o token Databricks é obtido do contexto do notebook para autenticação com serviços externos.

In [ ]:
from pyspark.sql import SparkSession
import json
import re
import os
import sys
import mlflow
import time
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
from typing import List, Any
import openai
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from databricks.feature_store import FeatureStoreClient
from pyspark.sql import Row



spark = SparkSession.builder.appName("LLM_Extractor").getOrCreate()

DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get() if dbutils.notebook.entry_point.getDbutils().notebook().getContext() is not None else None

## Desativação de Exibição MLflow no Notebook

Esta célula desativa a exibição automática de informações do MLflow no notebook, configurando o comportamento do MLflow para evitar que ele adicione visualizações e saídas extras durante o rastreamento de experimentos. Isto é particularmente útil em notebooks de produção onde queremos controlar precisamente o que é exibido, sem poluição visual extra criada pelo rastreamento automático do MLflow.

O comando `mlflow.tracing.disable_notebook_display()` evita que artefatos, métricas e parâmetros sejam automaticamente exibidos no notebook quando são registrados, mantendo a saída limpa e focada apenas no que está sendo explicitamente exibido pelo código.

In [ ]:
mlflow.tracing.disable_notebook_display()

## Configuração dos Parâmetros de Consulta

Esta célula define os parâmetros de consulta para extrair laudos de imunohistoquímica específicos para câncer de mama. São configurados:

1. **Tabela de Destino** (`table_imuno`): Define o nome da tabela Delta onde os resultados serão armazenados.

2. **Cláusula WHERE para Incremento** (`where_clause`): Implementa uma estratégia de carga incremental, buscando apenas registros com timestamp mais recente que o último carregamento na tabela de destino.

3. **Filtros de Extração** (`filtro_extracao`): Define critérios específicos para selecionar apenas laudos relevantes:
   - Linha de cuidado específica para mama
   - Pacientes do sexo feminino
   - Siglas de exame específicas de imunohistoquímica (IH-NEO, IHMAMA)
   - Presença dos termos "mama" e "carcinoma" nos laudos

Esses filtros garantem que apenas os laudos de imunohistoquímica relevantes para câncer de mama feminino sejam processados, otimizando o uso de recursos computacionais e do modelo de linguagem.

In [ ]:
table_imuno = "refined.saude_preventiva.fleury_laudos_mama_imunohistoquimico" 

where_clause = f"""
WHERE
    flr.`_datestamp` >= (
        SELECT MAX(imuno._datestamp)
        FROM {table_imuno} imuno
    )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado  = 'mama'
        AND UPPER(sexo_cliente) = 'F'
        AND sigla_exame IN ("IH-NEO", "IHMAMA")
        AND laudo_tratado RLIKE '(?i)mama' AND laudo_tratado RLIKE '(?i)carcinoma'
"""

## Execução da Consulta SQL e Carregamento dos Dados

Esta célula constrói e executa uma consulta SQL para extrair laudos de exames imunohistoquímicos relevantes para análise. 

**Processo detalhado:**

1. **Construção da Consulta**: A consulta SQL é construída usando um Common Table Expression (CTE) chamado `base` que:
   - Seleciona colunas relevantes da tabela principal de laudos (`refined.saude_preventiva.fleury_laudos`)
   - Aplica o filtro incremental definido em `where_clause` para obter apenas registros novos
   - Usa a cláusula `filtro_extracao` para filtrar apenas laudos de mama com carcinoma

2. **Seleção de Campos**: A consulta extrai informações essenciais como:
   - Identificadores (id_marca, id_unidade, id_cliente, id_ficha, etc.)
   - Datas (dth_pedido, dth_resultado)
   - Tipo de exame (sigla_exame)
   - Conteúdo do laudo (laudo_tratado)
   - Informações da linha de cuidado e sexo do cliente

3. **Execução da Consulta**: A consulta é executada através do Spark SQL, criando um DataFrame `df_spk`

4. **Visualização**: O DataFrame resultante é exibido na interface para verificação inicial dos dados carregados

Esta etapa é fundamental para preparar o conjunto de dados que será processado pelo modelo de linguagem nas etapas subsequentes.

In [ ]:
query = f"""
WITH base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`
    FROM refined.saude_preventiva.fleury_laudos flr    
    {where_clause}
)
SELECT * FROM base
{filtro_extracao}
"""
df_spk = spark.sql(query)
display(df_spk)

## Verificação de Sexo e Tipos de Exame

Esta célula executa uma verificação básica dos dados carregados, exibindo os valores distintos para duas colunas críticas: `sexo_cliente` e `sigla_exame`. 

O objetivo desta verificação é:
1. Confirmar que apenas pacientes do sexo feminino (F) foram incluídos no dataset, conforme especificado no filtro
2. Verificar quais siglas de exames de imunohistoquímica estão presentes nos dados carregados
3. Validar que os filtros da consulta SQL foram aplicados corretamente

Esta etapa de validação rápida ajuda a garantir a qualidade dos dados antes de prosseguir com análises mais detalhadas e o processamento pelo modelo de linguagem.

In [ ]:
display(df_spk.select("sexo_cliente", "sigla_exame").distinct())

## Contagem de Registros

Esta célula simples realiza a contagem do número total de registros no DataFrame `df_spk` carregado pela consulta SQL. O comentário "# 3635" indica que, em uma execução anterior, foram encontrados 3.635 registros.

Conhecer o volume de dados é importante para:
1. Estimar o tempo total de processamento pelo modelo de linguagem
2. Verificar se o volume está de acordo com o esperado
3. Avaliar a necessidade de processamento em lotes ou amostragem para testes

Esta contagem serve como referência para monitoramento do pipeline em execuções futuras.

In [ ]:
df_spk.count()
# 3635

## Funções para Processamento de Laudos com LLM

Esta célula define funções essenciais para o processamento de laudos médicos utilizando um modelo de linguagem (LLM). São definidas quatro funções principais:

### 1. `prompt_laudo`
Define o template do prompt que será enviado ao LLM, instruindo-o a extrair informações específicas do laudo:
- **Receptor de Estrogênio**: Status (POSITIVO/NEGATIVO)
- **Receptor de Progesterona**: Status (POSITIVO/NEGATIVO)
- **Status HER-2**: Classificado conforme escores (0/1+ como NEGATIVO, 2+ como INCONCLUSIVO, 3+ como POSITIVO)
- **Ki-67 (%)**: Valor numérico da porcentagem de células positivas
- **Status CK5/6**: Status (POSITIVO/NEGATIVO)

### 2. `generate`
Função principal que:
- Recebe um laudo e uma descrição do agente
- Formata o prompt com o conteúdo do laudo
- Configura parâmetros do modelo (temperatura=0 para determinismo)
- Trata erros de conexão com retentativas
- Retorna a resposta do modelo

### 3. `batch_generate`
Função para processamento em lote que:
- Divide os laudos em lotes menores
- Processa cada laudo utilizando a função `generate`
- Exibe uma barra de progresso usando `tqdm`
- Retorna uma lista com as respostas do modelo

### 4. `limpar_e_converter`
Função para pós-processamento que:
- Remove tags de código Markdown da resposta (```python, ```)
- Converte o texto em JSON para um dicionário Python
- Trata erros de conversão, fornecendo valores padrão

Estas funções formam o núcleo do sistema de extração de informações, transformando o texto não estruturado dos laudos em dados estruturados que podem ser analisados programaticamente.

In [ ]:
def prompt_laudo(laudo_texto: str) -> str:
    prompt = f"""A seguir está um laudo médico de mamografia, conforme indicado abaixo. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

    Laudo clínico:
    \"\"\"{laudo_texto}\"\"\"

    ### Critérios de extração:

    - **Receptor de Estrogênio**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Receptor de Progesterona**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Status do HER-2**: retorne se o Status do HER-2 é "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO". Com base no score seguindo as regras:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"

    - **Ki-67 (%)**: retorne o valor numérico da porcentagem de positividade do KI-67.

    - **Status do CK5/6**: retorne "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO" do Status do CK5/6.

    ### Saída esperada (dicionário Python válido):
    ```python
    {{
    "receptor_estrogeno": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "receptor_progesterona": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "status_her2": "POSITIVO" | "POSITIVO" | "INCONCLUSIVO" |  "NÃO INFORMADO",
    "ki67_percentual": float |  0,
    "status_ck5_6": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO"
    }}
    ```
    """
    return prompt.strip()

def generate(descricao_agente:str, laudo:str, llm_client) -> str:
    """
    Gera o resultado da análise de um laudo
    Params:
        descricao_agente: descricao do agente que a LLM representa (primeira mensagem enviada à LLM)
        prompt: prompt base que será utilizado para gerar a análise
        laudo: laudo a ser analisado (incluido dentro do prompt)
        llm_client: cliente da API da LLM
    Return:
        response_message: resposta da LLM
    """
    prompt = prompt_laudo(laudo)
    messages = [
        # Utilizamos o primeiro prompt para contextualizar o llm do que ele deve fazer. 
        # No exemplo utilizamos a abordagem Role, Task, Output, Prompting.
        # Mas sintam-se a vontade para alterar de acordo com a necessidade
        {
            "role": "system",
            "content": descricao_agente
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    model_params = {
        "model": "databricks-llama-4-maverick",
        "messages": messages,
        "temperature": 0,
        "max_tokens": 4000,
        "top_p": 0.75,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
    connection_retry = 0
    while connection_retry < 3:
        try:
            response = llm_client.chat.completions.create(**model_params)
            response_message = response.choices[0].message.content
            break
        # TODO: verificar se a excessao é de conexao
        except (ConnectionError, TimeoutError) as e:
            connection_retry += 1
            print("Sem reposta do modelo")
            print(str(e))
            print("Tentando novamente...")
            time.sleep(0.1)
        except Exception as e:
            raise e

    if connection_retry >= 3:
        response_message = ''
    
    return response_message


def batch_generate(descricao_agente, laudos, llm_client, batch_size=25):
    responses = []
    
    llm_client = openai.OpenAI(
        api_key=DATABRICKS_TOKEN,
        base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
    )
    
    # Dividir em lotes
    for i in range(0, len(laudos), batch_size):
        laudos_batch = laudos[i:i+batch_size]
        for laudo in tqdm(laudos_batch, desc=f"Processando lote {i//batch_size + 1}", total=len(laudos_batch)):
            responses.append(generate(descricao_agente, laudo, llm_client))
    
    return responses

def limpar_e_converter(item):
    try:
        item_limpo = re.sub(r"```(?:python)?", "", item).replace("```", "").strip()
        return json.loads(item_limpo)
    except Exception as e:
        print(f"Erro ao converter resposta: {e}")
        return {
            "receptor_estrogeno": "NÃO INFORMADO",
            "receptor_progesterona": "NÃO INFORMADO",
            "status_her2": "NÃO INFORMADO",
            "ki67_percentual": "NÃO INFORMADO",
            "status_ck5_6": "NÃO INFORMADO"
        }

## Funções de Extração Heurística (Pseudo-Gold)

Esta célula define um conjunto de funções para extração heurística de informações dos laudos médicos, utilizando expressões regulares. Estas extrações servem como um "pseudo-gold standard" para avaliar o desempenho do LLM.

### Funções de Extração Individual

1. **`extrai_receptor_estrogeno`**: Identifica o status do receptor de estrogênio
   - Reconhece padrões como "receptor de estrogênio: positivo"
   - Identifica abreviações como "ER+" e "ER-"

2. **`extrai_receptor_progesterona`**: Identifica o status do receptor de progesterona
   - Reconhece padrões como "receptor de progesterona: positivo" 
   - Identifica abreviações como "PR+" e "PR-"

3. **`extrai_status_her2`**: Determina o status do HER-2 baseado em escores
   - Processa padrões como "HER-2 ESCORE X+"
   - Mapeia escores 0/1+ como "NEGATIVO", 2+ como "INCONCLUSIVO", 3+ como "POSITIVO"

4. **`extrai_ki67_percentual`**: Extrai o valor percentual do Ki-67
   - Processa padrões como "Ki-67: 20%"
   - Converte para valor numérico (float)

5. **`extrai_status_ck5_6`**: Identifica o status do CK5/6
   - Reconhece padrões como "CK5/6 - POSITIVO"
   - Identifica abreviações como "CK5/6+" e "CK5/6-"

### Função de Avaliação

**`avalia_extracao_sem_ground_truth_imuno`**: Função chave que compara extrações do LLM com extrações heurísticas
   - Gera extrações heurísticas para todos os campos
   - Compara campo a campo os resultados do modelo vs. heurísticas
   - Produz métricas de avaliação por campo (acertou/não acertou)

Este sistema de avaliação permite medir a qualidade das extrações do LLM sem necessidade de anotação manual de dados, sendo uma ferramenta crucial para validação contínua do pipeline.

In [ ]:
import ast
#import ace_tools as tools

# ==============================================
# Funções de extração heurística (pseudo-gold) para o novo prompt
# ==============================================

def extrai_receptor_estrogeno(txt: str) -> str:
    """
    Extrai o status de Receptor de Estrogênio: valores possíveis
    "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """
    txt_lower = txt.lower()
    # Padrões comuns: "receptor de estrogênio: positivo" ou "er+: positivo" etc.
    if re.search(r"receptor\s+de\s+estrog[eê]genio\s*[:\-]?\s*positivo", txt_lower):
        return "POSITIVO"
    if re.search(r"receptor\s+de\s+estrog[eê]genio\s*[:\-]?\s*negativo", txt_lower):
        return "NEGATIVO"
    # Abreviações: "ER+" / "ER-"
    if re.search(r"\ber\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\ber\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"
    return "NÃO INFORMADO"

def extrai_receptor_progesterona(txt: str) -> str:
    """
    Extrai o status de Receptor de Progesterona: valores possíveis
    "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """
    txt_lower = txt.lower()
    if re.search(r"receptor\s+de\s+progesterona\s*[:\-]?\s*positivo", txt_lower):
        return "POSITIVO"
    if re.search(r"receptor\s+de\s+progesterona\s*[:\-]?\s*negativo", txt_lower):
        return "NEGATIVO"
    # Abreviações: "PR+" / "PR-"
    if re.search(r"\bpr\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bpr\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"
    return "NÃO INFORMADO"

def extrai_status_her2(txt: str) -> str:
    """
    Extrai o Status do HER-2 baseado em termos de score:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"
    """
    txt_lower = txt.lower()
    # Primeiro, busca padrão "her-2 ... escore X+"
    m = re.search(r"her[-\s]?2.*?escore\s*[:\-]?\s*([0-3]\+?)", txt_lower, flags=re.IGNORECASE)
    if m:
        score = m.group(1)
        if score.startswith("0") or score == "1+":
            return "NEGATIVO"
        if score == "2+":
            return "INCONCLUSIVO"
        if score == "3+":
            return "POSITIVO"
    # Outra forma: "her2 3+" isolado
    m2 = re.search(r"\bher[-]?2\s*[:\-]?\s*([0-3]\+)\b", txt_lower, flags=re.IGNORECASE)
    if m2:
        score2 = m2.group(1)
        if score2 == "1+":
            return "NEGATIVO"
        if score2 == "2+":
            return "INCONCLUSIVO"
        if score2 == "3+":
            return "POSITIVO"
    return "NÃO INFORMADO"

def extrai_ki67_percentual(txt: str) -> float:
    """
    Extrai o valor de Ki-67 em porcentagem. 
    Exemplo no texto: "Ki-67: 20%", "Ki 67 15 %", etc.
    Se não encontrar, retorna 0.0.
    """
    m = re.search(r"ki[-\s]?67.*?[:\-]?\s*(\d{1,3}(?:\.\d+)?)\s*%", txt, flags=re.IGNORECASE)
    if m:
        try:
            return float(m.group(1))
        except ValueError:
            return 0.0
    return 0.0

def extrai_status_ck5_6(txt: str) -> str:
    """
    Extrai o status do CK5/6: "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """
    txt_lower = txt.lower()
    if re.search(r"ck5\s*\/\s*6.*?[:\-]?\s*positivo", txt_lower):
        return "POSITIVO"
    if re.search(r"ck5\s*\/\s*6.*?[:\-]?\s*negativo", txt_lower):
        return "NEGATIVO"
    # Abreviações: "CK5/6+" ou "CK5/6-"
    if re.search(r"\bck5\s*\/\s*6\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bck5\s*\/\s*6\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"
    return "NÃO INFORMADO"

# ==============================================
# Função de avaliação sem ground truth completo (novo conjunto de campos)
# ==============================================

def avalia_extracao_sem_ground_truth_imuno(laudo_texto: str, json_modelo: dict):
    """
    Gera pseudo-gold (json_heu) para os campos do novo prompt
    e compara com json_modelo (saída da IA).
    Retorna:
      - json_heu: dicionário com valores heurísticos
      - comparacoes: dicionário que, para cada campo, traz:
          * valor_heu
          * valor_mod
          * acertou (boolean)
    """
    # 1. Gera pseudo-gold (json_heu)
    rei = extrai_receptor_estrogeno(laudo_texto)
    rpr = extrai_receptor_progesterona(laudo_texto)
    her = extrai_status_her2(laudo_texto)
    ki6 = extrai_ki67_percentual(laudo_texto)
    ck6 = extrai_status_ck5_6(laudo_texto)

    json_heu = {
        "receptor_estrogeno": rei,
        "receptor_progesterona": rpr,
        "status_her2": her,
        "ki67_percentual": ki6,
        "status_ck5_6": ck6
    }

    # 2. Prepara json_modelo: se não for dict, converte para dict vazio
    if not isinstance(json_modelo, dict):
        json_modelo = {}

    # 3. Comparações campo a campo
    comparacoes = {}

    # Campos categóricos (strings)
    for campo in ["receptor_estrogeno", "receptor_progesterona", "status_her2", "status_ck5_6"]:
        val_heu = json_heu[campo]
        val_mod = json_modelo.get(campo, "NÃO INFORMADO")
        comparacoes[campo] = {
            "valor_heu": val_heu,
            "valor_mod": val_mod,
            "acertou": (val_heu == val_mod)
        }

    # Campo Ki-67 (%)
    val_heu_ki = json_heu["ki67_percentual"]
    try:
        val_mod_ki = float(json_modelo.get("ki67_percentual", 0))
    except (ValueError, TypeError):
        val_mod_ki = 0.0
    comparacoes["ki67_percentual"] = {
        "valor_heu": val_heu_ki,
        "valor_mod": val_mod_ki,
        "acertou": (val_heu_ki == val_mod_ki)
    }

    return json_heu, comparacoes

## Função Auxiliar para Parsing de JSON

Esta célula define uma função de utilidade para lidar com o parsing de strings JSON. A função `parse_json_string` é responsável por converter strings que representam dicionários Python para objetos Python reais.

**Objetivo da Célula:**
- Fornecer uma função segura para converter strings que representam estruturas de dados Python em objetos Python efetivos
- Tratar graciosamente casos onde a conversão falha

**Detalhamento da Função:**
- Verifica se o input já é uma string (caso não seja, retorna o próprio input)
- Utiliza `ast.literal_eval` para converter a string em um objeto Python
- Se a conversão falhar (por erro de formato, por exemplo), retorna um dicionário vazio

Esta função é especialmente importante para garantir que as respostas do modelo de linguagem, que são retornadas como texto, possam ser tratadas como estruturas de dados Python para análise e validação posteriores.

In [ ]:
def parse_json_string(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except Exception:
            return {}
    return s

## Função de Agregação de Resultados

Esta célula define uma função especializada para agregar resultados de avaliação de múltiplos laudos, calculando métricas de desempenho para cada campo extraído.

**Objetivo da Função:**
`agrega_resultados_dinamico` processa uma lista de comparações (gerada pela função `avalia_extracao_sem_ground_truth_imuno`) e calcula estatísticas agregadas, incluindo:
- Número total de acertos por campo
- Total de laudos analisados
- Taxa de acerto (acertos/total) para cada campo

**Detalhamento da Implementação:**
1. Utiliza `Counter` para contabilizar os acertos por campo de forma eficiente
2. Itera por todas as comparações e todos os campos dentro de cada comparação
3. Verifica o atributo "acertou" de cada campo para contabilizar os acertos
4. Gera um dicionário estruturado com as estatísticas de cada campo
5. Garante que todos os campos estão representados no resultado, mesmo os que não tiveram acertos

**Características Importantes:**
- É uma função genérica que não assume um conjunto fixo de campos, podendo processar qualquer estrutura de comparações
- Trata adequadamente o caso de divisão por zero quando não há laudos
- Mantém consistência na estrutura de saída para facilitar análises posteriores

Esta função é crucial para avaliar quantitativamente o desempenho do modelo de extração em todo o conjunto de dados.

In [ ]:
from collections import Counter

def agrega_resultados_dinamico(lista_comparacoes):
    """
    Agraga resultados de uma lista de dicionários de comparações, retornando para cada campo:
      - acertos: número de vezes que 'acertou' == True
      - total: número total de laudos (len(lista_comparacoes))
      - taxa_acerto: acertos / total (ou 0.0 se total == 0)
    
    Suporta qualquer conjunto de chaves em cada dict, desde que cada valor seja outro dict contendo a chave 'acertou'.
    """
    total_laudos = len(lista_comparacoes)
    acertos_por_campo = Counter()

    # Para cada comparação, percorremos todas as chaves e contamos os acertos
    for comp in lista_comparacoes:
        for campo, info in comp.items():
            # Supondo que info seja um dict com a chave "acertou"
            if info.get("acertou", False):
                acertos_por_campo[campo] += 1

    # Monta o dicionário de saída
    resultado = {}
    for campo, acertos in acertos_por_campo.items():
        resultado[campo] = {
            "acertos": acertos,
            "total": total_laudos,
            "taxa_acerto": (acertos / total_laudos) if total_laudos > 0 else 0.0
        }

    # É possível que exista algum campo em alguma comparação que nunca teve 'acertou' == True.
    # Se quisermos incluir também esses campos (com acertos = 0), podemos varrer as chaves da primeira entrada:
    if lista_comparacoes:
        primeira = lista_comparacoes[0]
        for campo in primeira.keys():
            if campo not in resultado:
                resultado[campo] = {
                    "acertos": 0,
                    "total": total_laudos,
                    "taxa_acerto": 0.0
                }

    return resultado

## Verificação de Contagem de Registros

Esta célula executa uma verificação rápida de quantos registros estão disponíveis para processamento no DataFrame `df_spk`. 

Esta contagem é importante para:
1. Verificar se temos dados para processar
2. Dimensionar o tempo de execução esperado
3. Avaliar se o processo de filtragem nos retornou um conjunto de dados compatível com o esperado

O valor retornado será o número total de laudos de imunohistoquímica de mama com menção a carcinoma que estão disponíveis para processamento pelo modelo de linguagem.

In [ ]:
df_spk.count()

## Extração de Marcadores com LLM e Classificação Molecular

Esta célula implementa o core do pipeline de processamento dos laudos, envolvendo a extração de informações estruturadas dos laudos médicos utilizando um modelo de linguagem e a subsequente classificação molecular dos casos de câncer. Este é o ponto central do notebook, onde o processamento efetivo dos dados acontece.

**Objetivo:**
1. Configurar o cliente LLM para comunicação com o endpoint Databricks
2. Processar os laudos médicos para extração de biomarcadores 
3. Classificar os tumores em subtipos moleculares com base nos biomarcadores extraídos

**Fluxo de Processamento:**
1. **Inicialização do Cliente LLM**: Configura o cliente OpenAI para se comunicar com o endpoint Databricks
2. **Definição de Persona**: Define o papel do modelo como "médico oncologista especialista"
3. **Amostragem de Dados**: Limita a análise a 15 registros para processamento local
4. **Processamento em Lote**: Envia os laudos para o modelo em lotes
5. **Pós-processamento**: Converte as respostas do modelo em formato estruturado
6. **Classificação Molecular**: Aplica regras específicas para determinar o subtipo molecular:
   - **Luminal A**: ER+ ou PR+, HER2-, Ki-67 < 14%
   - **Luminal B**: ER+ ou PR+, HER2-, Ki-67 ≥ 14%
   - **Luminal com HER2 Positivo**: ER+ ou PR+, HER2+
   - **HER-2 Superexpresso**: ER-, PR-, HER2+
   - **Triplo Negativo**: ER-, PR-, HER2-

A célula contém também código comentado que representa versões anteriores ou alternativas do pipeline, incluindo opções para persistência dos dados e registro de métricas, que podem ser descomentadas conforme necessário.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, IntegerType
from pyspark.sql.functions import from_json, col
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import StringType
import pandas as pd

# realizar extração

if df_spk.count() > 0:
    llm_client = openai.OpenAI(api_key=DATABRICKS_TOKEN,
                           base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
                           )
    descricao_agente = "Atue como um médico oncologista especialista em laudos de mamografia."

    # Coleta os dados localmente
    df_local = df_spk.select("ficha","id_item","id_subitem","id_cliente","dth_pedido","dth_resultado", "sigla_exame", "laudo_tratado","linha_cuidado","_datestamp").limited(15).toPandas()
  

    # Aplica o LLM localmente
    df_local["resposta_llm"] = batch_generate(descricao_agente, df_local["laudo_tratado"].tolist(), llm_client, batch_size=100)
    df_local= df_local.join(df_local["resposta_llm"].apply(limpar_e_converter).apply(pd.Series)) 

    # Adiciona as respostas ao DataFrame local
    #df_local["resposta_llm"] = respostas_limpa

    # Converte de volta para Spark   
    df_respostas = spark.createDataFrame(df_local) 
    display(df_respostas)

    # # Faz join com o DataFrame original para manter todas as colunas
    # df_final = df_spk.join(df_respostas.select("ficha","id_item","id_subitem", "resposta_llm"), on=["ficha","id_item","id_subitem"], how="inner")

    # # Definir estrutura
    # schema_resposta = StructType([
    # StructField("receptor_estrogeno", StringType(), True),
    # StructField("receptor_progesterona", StringType(), True),
    # StructField("status_her2", StringType(), True),
    # StructField("ki67_percentual", DoubleType(), True),
    # StructField("status_ck5_6", DoubleType(), True),
    # ])

    # df_final_expanded = df_final.withColumn("resposta_struct", col("resposta_llm"))

    # # expandir resultado llma para colunas

    # df_final_expanded = df_final_expanded.select(
    # "*",
    # col("resposta_struct.receptor_estrogeno").alias("receptor_estrogeno"),
    # col("resposta_struct.receptor_progesterona").alias("receptor_progesterona"),
    # col("resposta_struct.status_her2").alias("status_her2"),
    # col("resposta_struct.ki67_percentual").alias("ki67_percentual"),
    # col("resposta_struct.status_ck5_6").alias("status_ck5_6"),
    # ).drop("resposta_struct")

    #display(df_final_expanded)

    # a partir dos dados extraídos definir uma classificação final
    df_final_classif = df_respostas.withColumn(
            "categoria_final",
            F.when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "NEGATIVO") &
                    (F.col("ki67_percentual") < 14)
                ),
                "Luminal A"
            ).when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "NEGATIVO") &
                    (F.col("ki67_percentual") >= 14)
                ),
                "Luminal B"
            ).when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "POSITIVO")
                ),
                "Luminal com HER2 Positivo"
            ).when(
                (
                    (F.col("receptor_estrogeno") == "NEGATIVO") &
                    (F.col("receptor_progesterona") == "NEGATIVO") &
                    (F.col("status_her2") == "POSITIVO")
                ),
                "HER-2 Superexpresso"
            ).when(
                (
                    (F.col("receptor_estrogeno") == "NEGATIVO") &
                    (F.col("receptor_progesterona") == "NEGATIVO") &
                    (F.col("status_her2") == "NEGATIVO")
                ),
                "Triplo Negativo"
            ).otherwise("Indefinido")
        )

    display(df_final_classif)

    # schema_resposta = StructType([
    # StructField("receptor_estrogeno", StringType(), True),
    # StructField("receptor_progesterona", StringType(), True),
    # StructField("status_her2", StringType(), True),
    # StructField("ki67_percentual", StringType(), True),
    # StructField("status_ck5_6", StringType(), True),
    # ])

    # df_final_expanded = df_final.withColumn("resposta_struct", from_json(col("resposta_llm"), schema_resposta))

    # df_final_expanded = df_final_expanded.select(
    # "*",
    # col("resposta_struct.receptor_estrogeno").alias("receptor_estrogeno"),
    # col("resposta_struct.receptor_progesterona").alias("receptor_progesterona"),
    # col("resposta_struct.status_her2").alias("status_her2"),
    # col("resposta_struct.ki67_percentual").alias("ki67_percentual"),
    # col("resposta_struct.status_ck5_6").alias("status_ck5_6"),
    # ).drop("resposta_llm", "resposta_struct")

    # display(df_final_expanded) 


    # ###################### Apenas para testes #################
    # df_imunohistoquimico = df_imunohistoquimico.limit(100)
    # ###########################################################

    # rows = df_spk.select("laudo_tratado").collect()
    # laudos = [row.laudo_tratado for row in rows]

    # respostas = batch_generate(descricao_agente, laudos, llm_client, batch_size=300)

    # lista_dicts = [limpar_e_converter(item) for item in respostas]

    # schema = StructType([
    #     StructField("receptor_estrogeno", StringType(), True),
    #     StructField("receptor_progesterona", StringType(), True),
    #     StructField("status_her2", StringType(), True),
    #     StructField("ki67_percentual", StringType(), True),
    #     StructField("status_ck5_6", StringType(), True),
    # ])

    # df_lista = spark.createDataFrame(lista_dicts, schema=schema)


    # w = Window.orderBy(F.lit(1))

    # df_imuno_indexed = (
    #     df_imunohistoquimico
    #     .withColumn("row_id", F.row_number().over(w) - 1)  # subtrai 1 para ficar zero‐based
    # )

    # df_lista_indexed = (
    #     df_lista
    #     .withColumn("row_id", F.row_number().over(w) - 1)
    # )

    # df_final = df_imuno_indexed.join(df_lista_indexed, on="row_id").drop("row_id")
    # df_final = df_final.withColumn(

    # df_final_classif = df_spk.withColumn(
    #     "categoria_final",
    #     F.when(
    #         (
    #             ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    #             (F.col("status_her2") == "NEGATIVO") &
    #             (F.col("ki67_percentual") < 14)
    #         ),
    #         "Luminal A"
    #     ).when(
    #         (
    #             ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    #             (F.col("status_her2") == "NEGATIVO") &
    #             (F.col("ki67_percentual") >= 14)
    #         ),
    #         "Luminal B"
    #     ).when(
    #         (
    #             ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    #             (F.col("status_her2") == "POSITIVO")
    #         ),
    #         "Luminal com HER2 Positivo"
    #     ).when(
    #         (
    #             (F.col("receptor_estrogeno") == "NEGATIVO") &
    #             (F.col("receptor_progesterona") == "NEGATIVO") &
    #             (F.col("status_her2") == "POSITIVO")
    #         ),
    #         "HER-2 Superexpresso"
    #     ).when(
    #         (
    #             (F.col("receptor_estrogeno") == "NEGATIVO") &
    #             (F.col("receptor_progesterona") == "NEGATIVO") &
    #             (F.col("status_her2") == "NEGATIVO")
    #         ),
    #         "Triplo Negativo"
    #     ).otherwise("INDEFINIDO")
    # )

    # df_final = (df_final.withColumn("id_unidade", F.col("id_unidade").cast(LongType()))
    #                     .withColumn("id_cliente", F.col("id_cliente").cast(LongType()))
    #                     .withColumn("id_item", F.col("id_item").cast(LongType()))
    #                     .withColumn("id_subitem", F.col("id_subitem").cast(LongType()))
    #                     .withColumn("id_exame", F.col("id_exame").cast(LongType()))
    #                     .withColumn("index",            F.col("index").cast(LongType()))
    #                 )

    # Base histórica
    #fs = FeatureStoreClient()
    #fs.create_table(
    #    name="refined.saude_preventiva.fleury_laudos_mamo_imunohistoquimico",
    #    primary_keys=["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"],
    #    schema=df_final.schema,
    #    description="Features extraídas de laudos de mamografia. Siglas: IH-NEO e IHMAMA"
    #)

    # Append em prd
    # num_linhas = df_final.count()
    # fs = FeatureStoreClient()
    # if num_linhas > 0:
    #     print(f"Há {num_linhas} registros para inserir — executando gravação…")
    #     primary_keys = ["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"]
    #     ###### Apenas para testes ##############
    #     df_final = df_final.dropna()
    #     df_final = df_final.dropDuplicates(primary_keys)
    #     ########################################
    #     df_final = df_final.dropDuplicates(primary_keys)
    #     fs.write_table(
    #         name="refined.saude_preventiva.fleury_laudos_mamo_imunohistoquimico",
    #         df=df_final,
    #         mode="merge",
    #     )
    # else:
    #     print("Nenhum registro encontrado; nada a fazer.")

    # df_metrics = pd.DataFrame()
    # df_metrics["laudos"] = laudos
    # df_metrics["resultados"] = lista_dicts

    # df_metrics["resultados"] = df_metrics["resultados"].apply(parse_json_string)

    # lista_laudos2 = df_metrics["laudos"].tolist()
    # lista_modelo2 = df_metrics["resultados"].tolist()

    # lista_pseudo_gold2 = []
    # lista_comparacoes2 = []

    # for laudo_txt, json_mod in zip(lista_laudos2, lista_modelo2):
    #     json_heu, comp = avalia_extracao_sem_ground_truth_novo(laudo_txt, json_mod)
    #     lista_pseudo_gold2.append(json_heu)
    #     lista_comparacoes2.append(comp)

    # df_metrics = pd.DataFrame()
    # df_metrics["laudos"] = laudos
    # df_metrics["resultados"] = lista_comparacoes2
    # resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
    # df_metrics = pd.concat(
    #         [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
    #         axis=1
    #     )
    
    # json_metricas = agrega_resultados_dinamico(lista_comparacoes2)

    # from octoops import mlflowManager
    # #  mlflow.set_experiment("/Users/aureliano.paiva@grupofleury.com.br/imunohistoquimico_fleury_metricas")
    # experiment_id = mlflowManager.get_or_create_experiment("/Shared/saude_preventiva_mama/experiments_imunohistoquimico")
    # mlflowManager.enable_autologging()   

    # threshold = 0.8

    # with mlflowManager.start_run(run_name="Extracao_Laudos_Run_Threshold"):
    #     for campo, stats in json_metricas.items():
    #         taxa = stats["taxa_acerto"]
            
    #         # Registrar a taxa de acerto
    #         mlflowManager.log_metric(f"{campo}_taxa_acerto", taxa)
            
    #         # Registrar flag de aprovação no threshold
    #         passou_flag = 1 if taxa >= threshold else 0
    #         mlflowManager.log_metric(f"{campo}_passou_threshold", passou_flag)
            
    #         # Opcional: registrar acertos e total
    #         mlflowManager.log_metric(f"{campo}_acertos", stats["acertos"])
    #         mlflowManager.log_metric(f"{campo}_total", stats["total"])
        
    #     run_id = mlflowManager.active_run().info.run_id
    #     print(f"Run registrada: {run_id}")

## Processamento de Métricas de Avaliação (Código de Referência)

Esta célula contém código de referência para processar métricas de avaliação do desempenho do modelo de extração. Embora não seja executado diretamente (pois depende de variáveis não definidas no fluxo principal), este código serve como template para avaliação de qualidade.

O código demonstra como:
1. Criar um DataFrame de métricas com laudos e resultados
2. Converter estruturas de dados em formatos apropriados
3. Executar a função de avaliação para cada laudo
4. Transformar os resultados para facilitar análise

Este tipo de avaliação é útil para monitorar a qualidade das extrações e identificar áreas para melhoria do prompt ou do processamento.

In [ ]:
    df_metrics = pd.DataFrame()
    df_metrics["laudos"] = laudos
    df_metrics["resultados"] = lista_dicts

    df_metrics["resultados"] = df_metrics["resultados"].apply(parse_json_string)

    lista_laudos2 = df_metrics["laudos"].tolist()
    lista_modelo2 = df_metrics["resultados"].tolist()

    lista_pseudo_gold2 = []
    lista_comparacoes2 = []

    for laudo_txt, json_mod in zip(lista_laudos2, lista_modelo2):
        json_heu, comp = avalia_extracao_sem_ground_truth_imuno(laudo_txt, json_mod)
        lista_pseudo_gold2.append(json_heu)
        lista_comparacoes2.append(comp)

    df_metrics = pd.DataFrame()
    df_metrics["laudos"] = laudos
    df_metrics["resultados"] = lista_comparacoes2
    resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
    df_metrics = pd.concat(
            [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
            axis=1
        )
    
    json_metricas = agrega_resultados_dinamico(lista_comparacoes2)

## Visualização dos Resultados do Modelo

Esta célula realiza uma visualização direta do dicionário `json_mod`, que contém os resultados da extração do modelo de linguagem para um laudo específico. A função `display()` formata os dados para visualização interativa no ambiente Databricks.

Esta visualização permite inspecionar rapidamente os valores extraídos para um laudo individual, servindo como uma verificação rápida da qualidade das extrações. Os campos exibidos incluem os valores para cada um dos biomarcadores: receptor de estrogênio, receptor de progesterona, status HER2, percentual do Ki-67 e status do CK5/6.

In [ ]:
display(json_mod)

## Visualização das Métricas de Avaliação

Esta célula exibe o DataFrame `df_metrics`, que contém os resultados detalhados da avaliação das extrações do modelo em comparação com as extrações heurísticas (pseudo-gold).

O DataFrame exibe:
1. Os laudos médicos originais
2. Para cada campo extraído (receptores de estrogênio, progesterona, status HER2, etc.):
   - O valor extraído pelo método heurístico
   - O valor extraído pelo modelo de linguagem
   - Um indicador booleano de acerto (True/False)

Esta visualização é fundamental para análise detalhada do desempenho do modelo campo a campo e laudo a laudo, permitindo identificar padrões de erros específicos e oportunidades de melhoria.

In [ ]:
display(df_metrics)	

## Cálculo de Métricas de Avaliação

Esta célula implementa o processo de avaliação da qualidade das extrações do LLM para o conjunto de laudos processados. O objetivo é comparar sistematicamente os resultados do modelo com as extrações heurísticas (pseudo-gold) para cada campo.

**Fluxo de processamento:**
1. Coleta todos os laudos processados do DataFrame final classificado
2. Inicializa uma lista vazia para armazenar os resultados das comparações
3. Para cada laudo:
   - Obtém o texto original do laudo e a resposta do LLM
   - Aplica a função de limpeza e conversão à resposta do LLM (se necessário)
   - Executa a função `avalia_extracao_sem_ground_truth_imuno` para comparar com extrações heurísticas
   - Armazena os resultados da comparação

4. Cria um DataFrame para análise com:
   - Os laudos originais
   - Os resultados da avaliação normalizada (via `pd.json_normalize`)

5. Agrega os resultados usando a função `agrega_resultados_dinamico` para obter métricas consolidadas

Esta abordagem permite uma avaliação quantitativa da qualidade das extrações sem necessidade de anotação manual, utilizando regras heurísticas como referência.

In [ ]:
lista_laudos = df_final_classif.collect()
resultados = []
for row in lista_laudos:
    laudo_txt = row["laudo_tratado"]
    resposta_llm = row["resposta_llm"]
    # Corrige: só chama limpar_e_converter se for string, senão usa o dict diretamente
    if isinstance(resposta_llm, dict):
        json_mod = resposta_llm
    else:
        json_mod = limpar_e_converter(resposta_llm)
    pseudo_gold, compar = avalia_extracao_sem_ground_truth_imuno(laudo_txt, json_mod)
    resultados.append(compar)

df_metrics = pd.DataFrame()
df_metrics["laudos"] = df_respostas.select("laudo_tratado").toPandas()["laudo_tratado"]
df_metrics["resultados"] = resultados
resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
df_metrics = pd.concat(
    [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
    axis=1
)

json_metricas = agrega_resultados_dinamico(resultados)

## Visualização das Métricas Agregadas

Esta célula exibe o dicionário `json_metricas`, que contém as métricas agregadas de desempenho do modelo para cada campo extraído. 

As métricas incluem:
1. **Número de acertos**: Quantas vezes a extração do LLM concordou com a extração heurística
2. **Total de laudos**: Número total de laudos avaliados
3. **Taxa de acerto**: Proporção de acertos (acertos/total)

Esta visualização oferece uma visão consolidada do desempenho do modelo por campo, permitindo identificar quais campos têm melhor ou pior desempenho. Tais informações são cruciais para direcionar esforços de melhoria, como ajustes no prompt, refinamento das heurísticas de avaliação ou treinamento adicional.

In [ ]:
display(json_metricas)

## Contagem de Registros Classificados

Esta célula realiza uma contagem simples do número de registros no DataFrame `df_final_classif`, que contém os laudos processados com suas respectivas classificações moleculares.

Esta contagem serve para:
1. Verificar quantos laudos foram processados com sucesso
2. Confirmar que o processamento ocorreu como esperado
3. Fornecer informação importante para monitoramento do pipeline

O número retornado representa a quantidade total de laudos de imunohistoquímica de mama que foram classificados em subtipos moleculares pelo pipeline.

In [ ]:
df_final_classif.count()

## Persistência dos Dados Processados em Delta Lake

Esta célula final implementa a persistência dos dados processados em uma tabela Delta Lake, para uso posterior em análises e modelos preditivos. Utiliza o padrão de operação "merge" (upsert) para garantir idempotência e evitar duplicações.

**Componentes principais:**
1. **Configurações de Monitoramento**: Definição do webhook para alertas via Sentinel
2. **Inicialização da Sessão Spark**: Configuração da sessão para operações com Delta
3. **Definição do Caminho de Saída**: Especificação da tabela de destino
4. **Função `insert_data`**: Encapsula a lógica de inserção/atualização:
   - Verifica se a tabela Delta já existe
   - Cria a tabela se necessário
   - Executa uma operação de merge quando a tabela já existe
   
5. **Tratamento de Erros**: Estrutura try/except com:
   - Verificação da existência de dados para inserção
   - Chamada à função de inserção quando há dados
   - Envio de notificação via Sentinel quando não há dados para processamento
   - Captura e relato de exceções

Esta persistência é essencial para que os dados estruturados extraídos dos laudos e as classificações moleculares possam ser utilizados por sistemas downstream, como dashboards de BI, modelos preditivos ou ferramentas de apoio à decisão clínica.

In [ ]:
import traceback
from octoops import Sentinel
from delta.tables import DeltaTable

WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

# Iniciar sessão Spark
spark = SparkSession.builder.appName("DfPandasparaSpark").getOrCreate()

#OUTPUT_DATA_PATH = dbutils.widgets.get("OUTPUT_DATA_PATH")
OUTPUT_DATA_PATH = "refined.saude_preventiva.fleury_laudos_mama_imunohistoquimico"

# função para salvar dados na tabela
def insert_data(df_spk, output_data_path):

    # Cria a tabela Delta se não existir
    if not DeltaTable.isDeltaTable(spark, output_data_path):
        df_spk.write.format("delta").saveAsTable(output_data_path)
    else:
        # Carrega a tabela Delta existente
        delta_table = DeltaTable.forPath(spark, output_data_path)

        # Faz o merge (upsert)
        (delta_table.alias("target")
        .merge(
            df_spk.alias("source"),
            "target.ficha = source.ficha AND target.id_item = source.id_item AND target.id_subitem = source.id_subitem"
        )
        .whenMatchedUpdateAll() #atualiza todos os campos se o ID já existir
        .whenNotMatchedInsertAll() #insere se o ID não existir
        .execute())

 
# salvar dados na tabela
try:
    # 1/0
    if (df_final_classif.count() > 0):        


        # Inserir tabela catalog
        insert_data(df_final_classif, OUTPUT_DATA_PATH)
        print('Total de registros salvos na tabela:', df_final_classif.count())
       

    else: 
        error_message = traceback.format_exc()
        error_message = "Fleury Imunuhistoquimico - Não há laudos para extração."
        sentinela_ds_ai_business = Sentinel(
            project_name='Monitor_Linhas_Cuidado_Mama',
            env_type=WEBHOOK_DS_AI_BUSINESS_STG,
            task_title='Fleury Mama Imunuhistoquimico'
        )

        sentinela_ds_ai_business.alerta_sentinela(
            categoria='Alerta', 
            mensagem=error_message,
            job_id_descritivo='4_fleury_mama_Imunuhistoquimico'
        )
except Exception as e:
    traceback.print_exc()
    raise e    